## This notebook contains prediction part

#### run code in common notebook

In [2]:
%run 0_common.ipynb

2023-08-13 02:06:19.305023: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


7
<class 'list'>
['NDVI', 'Lai', 'SoilMoi00_10cm_tavg', 'LST_Day_1km', 'total_precipitation', 'u_component_of_wind_10m', 'v_component_of_wind_10m', 'T21']


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

### Predict on evaluation dataset

In [3]:
import numpy as np
batch_size = 64


ev = evaluation.take(100)
# Predict on the evaluation dataset with the specified number of steps
predictions = m.predict(x=ev)

print(len(predictions[0]))
print(len(predictions[0][0]))

NameError: name 'evaluation' is not defined

In [ ]:
feature_img_lists = []
date_start_str = '2019-07-01'
pred_start = ee.Date(date_start_str)
pred_end = pred_start.advance(4, 'week')

for feature in feature_list:
  img_col = getImageCollection(feature['name'],pred_region_geometry, feature['band'])
  pred_img = img_col.filterDate(pred_start, pred_end).median().unitScale(feature['min'], feature['max'])
  feature_img_lists += [pred_img]
print(feature_img_lists)

[<ee.image.Image object at 0x7fc7845536d0>, <ee.image.Image object at 0x7fc784553040>, <ee.image.Image object at 0x7fc7846e7580>, <ee.image.Image object at 0x7fc7846f8430>, <ee.image.Image object at 0x7fc78459c790>, <ee.image.Image object at 0x7fc78459cf70>, <ee.image.Image object at 0x7fc78459d5d0>]


In [ ]:
predict_image = feature_img_lists[0]
for idx in range(1, len(feature_img_lists)):
  # Combine all bands sampled from the same time to featureStack
  predict_image = ee.Image.cat([predict_image, feature_img_lists[idx]]).float()
# predict_image

In [ ]:
def doExport(predict_image, out_file_name, kernel_buffer, region):
    """Run the image export task.  Block until complete.
    """
    # Define the export task using the Earth Engine Python API.
    task = ee.batch.Export.image.toCloudStorage(
        image=predict_image,  # Select specific bands from the 'featureStack' Image.
        description=out_file_name,  # Description for the export task.
        bucket=OUTPUT_BUCKET,  # The destination Cloud Storage bucket.
        fileNamePrefix=FOLDER + '/' + out_file_name,  # File name prefix for the exported images.
        region=region.getInfo()['coordinates'],  # The bounding region for the export task.
        scale=SAMPLE_RESOLUTION,  # The spatial resolution in meters for the exported images.
        fileFormat='TFRecord',  # File format for the exported images (TensorFlow Record).
        maxPixels=1e10,  # Maximum number of pixels allowed for the export task.
        formatOptions={
            'patchDimensions': KERNEL_SHAPE,  # Dimensions of the image patches for segmentation.
            'kernelSize': kernel_buffer,  # Size of the kernel buffer for segmentation.
            'compressed': True,  # Enable compression for the exported images.
            'maxFileSize': 104857600  # Maximum file size for each exported image in bytes.
        }
    )
    print(FOLDER + '/' + out_file_name)
    # Start the export task.
    task.start()

    # Block until the task completes.
    print('Running image export to Cloud Storage...')
    import time
    while task.active():
        print('Polling for task (id: {}).'.format(task.id))
        time.sleep(30)

    # Check if the export task encountered an error.
    if task.status()['state'] != 'COMPLETED':
        print(f"Error with image export {task.status()['state']}")
    else:
        print('Image export completed.')

In [ ]:
doExport(predict_image, out_file_name, KERNEL_SIZE, pred_region)

**!remember to change user_folder to your own GEE username**

In [ ]:
# def doPrediction(out_image_base, in_image_bucket, out_image_bucket, user_folder, kernel_buffer):
#   """Perform inference on exported imagery, upload to Earth Engine.
#   """

#   print('Looking for TFRecord files...')

#   # Get a list of all the files in the output bucket.
#   filesList = !gsutil ls 'gs://'{in_image_bucket}'/'{FOLDER}

#   # Get only the files generated by the image export.
#   exportFilesList = [s for s in filesList if out_image_base in s]

#   # Get the list of image files and the JSON mixer file.
#   imageFilesList = []
#   jsonFile = None
#   for f in exportFilesList:
#     if f.endswith('.tfrecord.gz'):
#       imageFilesList.append(f)
#     elif f.endswith('.json'):
#       jsonFile = f

#   # Make sure the files are in the right order.
#   imageFilesList.sort()

#   from pprint import pprint
#   pprint(imageFilesList)
#   print(jsonFile)

#   import json
#   # Load the contents of the mixer file to a JSON object.
#   jsonText = !gsutil cat {jsonFile}
#   # Get a single string w/ newlines from the IPython.utils.text.SList
#   mixer = json.loads(jsonText.nlstr)
#   pprint(mixer)
#   patches = mixer['totalPatches']

#   # Get set up for prediction.
#   x_buffer = int(kernel_buffer[0] / 2)
#   y_buffer = int(kernel_buffer[1] / 2)

#   buffered_shape = [
#       KERNEL_SHAPE[0] + kernel_buffer[0],
#       KERNEL_SHAPE[1] + kernel_buffer[1]]

#   imageColumns = [
#     tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
#       for k in BANDS
#   ]

#   imageFeaturesDict = dict(zip(BANDS, imageColumns))

#   def parse_image(example_proto):
#     return tf.io.parse_single_example(example_proto, imageFeaturesDict)

#   def toTupleImage(inputs):
#     inputsList = [inputs.get(key) for key in BANDS]
#     stacked = tf.stack(inputsList, axis=0)
#     stacked = tf.transpose(stacked, [1, 2, 0])
#     return stacked

#    # Create a dataset from the TFRecord file(s) in Cloud Storage.
#   imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
#   imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
#   imageDataset = imageDataset.map(toTupleImage).batch(1)

#   # Perform inference.
#   print('Running predictions...')
#   predictions = m.predict(imageDataset, steps=patches, verbose=1)
#   # print(predictions[0])

#   print('Writing predictions...')
#   out_image_file = 'gs://' + out_image_bucket + '/' + FOLDER + '/' + out_image_base  + '.TFRecord'
#   writer = tf.io.TFRecordWriter(out_image_file)
#   patches = 0
#   for predictionPatch in predictions:
#     print('Writing patch ' + str(patches) + '...')
#     predictionPatch = predictionPatch[
#         x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

#     # Create an example.
#     example = tf.train.Example(
#       features=tf.train.Features(
#         feature={
#           'impervious': tf.train.Feature(
#               float_list=tf.train.FloatList(
#                   value=predictionPatch.flatten()))
#         }
#       )
#     )
#     # Write the example.
#     writer.write(example.SerializeToString())
#     patches += 1

#   writer.close()

#   # Start the upload.
#   out_image_asset = user_folder + '/' + out_image_base 
#   !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

for debug(change to your own username)

In [ ]:
user_folder = 'users/jindacz' # INSERT YOUR FOLDER HERE.
pred_kernel_buffer = [128, 128]
# date_start_str = '2019-07-01'
out_file_name = f'pred_Hanceville_{date_start_str}'
pred_region = ee.Geometry.Polygon(
    [[[-120.09501953125, 53.81667309827828], 
    [-120.09501953125, 56.69527587549462], 
    [-125.76396484375, 56.69527587549462],     
    [-125.76396484375, 53.81667309827828]]],  None, False)
FOLDER = 'fcnn-demo'

def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{OUTPUT_BUCKET}'/'{FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(f'imageFilesList {imageFilesList}')
  # print(f'jsonFile {jsonFile}')

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # print(f'jsonText{jsonText}')
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  # out_image_base = 'shufeitest123'
  out_image_file = 'gs://' + OUTPUT_BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  
  print(out_image_file)
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
    
  for predictionPatch in predictions:
    
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]
    
    # print(f'prediction patch {predictionPatch}')
    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    print('finish writing patch ' + str(patches) + '...')
    patches += 1


  #writer.close()
  print(f'fin writing')

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

# Run the prediction.
doPrediction(out_file_name, user_folder, pred_kernel_buffer, pred_region)

Looking for TFRecord files...
('imageFilesList '
 "['gs://6140-output-bucket/fcnn-demo/pred_Hanceville_2019-07-01.tfrecord.gz']")
{'patchDimensions': [256, 256],
 'patchesPerRow': 2,
 'projection': {'affine': {'doubleMatrix': [0.008983152841195215,
                                            0.0,
                                            -125.76413977673302,
                                            0.0,
                                            -0.008983152841195215,
                                            56.728610192147784]},
                'crs': 'EPSG:4326'},
 'totalPatches': 2}
Running predictions...


2023-08-12 20:17:00.729104: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-08-12 20:17:00.953701: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: v_component_of_wind_10m.  Can't parse serialized Example.
2023-08-12 20:17:00.953770: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: Key: v_component_of_wind_10m.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
2023-08-12 20:17:00.972742: W tensorflow/core/framework/op_kernel.cc:1830] OP_RE

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: v_component_of_wind_10m.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext]

In [ ]:
# !python -m pip install tensorflow-io 
# !python -m pip uninstall -y tensorflow-io

In [ ]:
# doPrediction(out_file_name, OUTPUT_BUCKET, DATA_BUCKET, FOLDER, kernel_size)

In [ ]:
# # Run the prediction.
print(user_folder)
print(out_file_name)

In [ ]:
# out_image = ee.Image(user_folder + '/' + bj_image_base)
out_image = ee.Image(user_folder + '/' + out_file_name)
mapid = out_image.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[53.81667309827828, -120.09501953125])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted impervious',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

export prediction to CSV

In [ ]:
pred_values = out_image.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=pred_region,
    scale=926.625433055833,  # The scale from the image's 'crs_transform'
)

impervious_list = pred_values.get('impervious').getInfo()

# Convert the list of pixel values to a pandas DataFrame
df = pd.DataFrame(impervious_list, columns=['prediction'])

# Save the DataFrame as a CSV file
df.to_csv('prediction_results.csv', index=False)

In [ ]:
print(pred_values.get('impervious'))